In [25]:
import pandas as pd
df = pd.read_csv("./System-Threat-Forecaster/train.csv")
df_copy = df.copy()

Q1.Identify the redundant columns in the training dataset based on the values taken. *

In [26]:
# Find constant columns
constant_columns = df.nunique()[df.nunique() == 1]
print(constant_columns)


IsBetaUser                     1
AutoSampleSubmissionEnabled    1
IsFlightsDisabled              1
dtype: int64


Q2.Determine the columns among the given set which have the highest positive correlation.

In [27]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

categorical_cols = df_copy.select_dtypes(include=['object']).columns

encoder = LabelEncoder()
for col in categorical_cols:
    df_copy[col] = encoder.fit_transform(df_copy[col])
corr_matrix = df_copy.corr()
corr_pairs = corr_matrix.abs().unstack()
corr_pairs =corr_pairs[corr_pairs < 1].sort_values(ascending=False)
corr_pairs.head(1)




OSEdition  OSSkuFriendlyName    0.998856
dtype: float64

Q3.Create a new dataframe called cat_df which contains only the columns of datatype  'object'. In cat_df, for all the columns which take less than or equal to 10 unique values, use OneHotEncoder(sparse=False, drop='first', handle_unknown='ignore'). What is the new number of columns in the cat_df dataframe?

In [38]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

cat_df = df.select_dtypes(include=['object'])
low_cardinality_cols = cat_df.nunique()[cat_df.nunique() <= 10].index

encoder = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')
encoded_cols = encoder.fit_transform(cat_df[low_cardinality_cols])

encoded_df = pd.DataFrame(encoded_cols, columns=encoder.get_feature_names_out(low_cardinality_cols))

cat_df = cat_df.drop(columns=low_cardinality_cols)
cat_df = pd.concat([cat_df, encoded_df], axis=1)
cat_df.shape[1]


82

Q4.Create a new dataframe called num_df which contains only the columns of datatype  'int64' and 'float64'. Use MinMaxScaler() on num_df. What is the sum of all the values in num_df?

In [45]:
from sklearn.preprocessing import MinMaxScaler

num_df = df.select_dtypes(include=['int64', 'float64'])

scaler = MinMaxScaler()
scaled_num_df = pd.DataFrame(scaler.fit_transform(num_df), columns=num_df.columns)
sum_values = scaled_num_df.sum().sum()
print(sum_values)


1585115.729433082


Use the original data (X,y) for this question. Apply the following steps on X:
1. Use SimpleImputer to fill missing values with strategy as 'most_frequent'
2. For all columns of datatype 'object', use OrdinalEncoder()
3. Apply train_test_split with test_size = 0.2 and random_state = 42
4. Train SGDClassifier with default parameters and random_state = 42

What is the accuracy score received on the test data from the train_test_split?
*


In [46]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

imputer = SimpleImputer(strategy='most_frequent')
df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

cat_cols = df.select_dtypes(include=['object']).columns
encoder = OrdinalEncoder()
df[cat_cols] = encoder.fit_transform(df[cat_cols])

X = df.drop(columns=['target'])  # Replace 'target' with the actual target column name
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

sgd_model = SGDClassifier(random_state=42)
sgd_model.fit(X_train, y_train)

y_pred = sgd_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)





Test Accuracy: 0.4951
